In [1]:
import numpy as np
import os
import pandas as pd
import SimpleFunctions as SF
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
# Grid
NPoints = 11
Nus     = np.linspace(0.395,0.455,NPoints+1)
Mus     = np.linspace(0.2,0.8,NPoints+1)
Alphas  = np.linspace(-30,-20,NPoints+1)


# Optimization Parameters
ConstitutiveModels = ['Ogden', 'Neo-Hookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Fixed']
BCsType  = BCsTypes[1]

NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

LoadCases = ['Compression', 'Tension', 'SimpleShear']
RelativeWeights = [1,1,1]


# Simulation parameters
FinalRelativeStretch = 0.1
RelativeStepSize     = 0.02


# Folder for the results
Output_Path = os.path.join('CostFunctionShape', BCsType, ConstitutiveModel)
os.makedirs(Output_Path, exist_ok=True)

FileName = str(NumberElements) + 'Elements' + str(NPoints) + '³Points.csv'

# Data frame to store values
Columns = ['Nu','Mu', 'Alpha', 'TotalCost']
DataFrame = pd.DataFrame(columns=Columns, index=range(int((NPoints+1)**3)), dtype='float64')

IterationTime = []

# Compute the map
Point = 0
for Nu in range(NPoints+1):
    for Mu in range(NPoints+1):        
        for Alpha in range(NPoints+1):
            
            # Initialize time tracking
            start = time.time()
            
            Parameters = [Nus[Nu], Mus[Mu], Alphas[Alpha]]
            
            TotalCost = SF.CostFunction(Parameters, ConstitutiveModel, BCsType, NumberElements, LoadCases, RelativeWeights, FinalRelativeStretch, RelativeStepSize, Plot = False)
            
            # Save values to table
            Parameters.append(TotalCost)
            DataFrame.loc[Point] = Parameters
            Point += 1
            
            IterationTime.append(time.time() - start)

# Save table
DataFrame.to_csv(os.path.join(Output_Path, FileName), index=False, sep=',')

Cost: 0.6728551739120215
Cost: 0.6973165569096245
Cost: 0.7211306138953143
Cost: 0.7442114184038816
Cost: 0.7664974021122039
Cost: 0.7879474509153912
Cost: 0.8085376786457458
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Cost: 0.8282588644111165
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Cost: 0.8471143770327206
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Cost: 0.8651186714256637
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time

In [ ]:
IterationTimes = np.array([159.205, 169.551])
print('Compilation time: %1.3f'%(IterationTimes[1]-IterationTimes[0]))
IterationTimes = np.array([159.205, 169.551, 168.555, 166.909, 165.516])
print('Mean Iteration time: %1.3f'%np.mean(IterationTimes))
IterationTime = np.array([159.205-158.716, 159.205-157.264, 159.205-155.129])
print('Mean time for new mesh generation: %1.3f'%np.mean(IterationTime))
a = (NPoints+1)**3*np.mean(IterationTimes)/(3600*24)
print('TotalTimeEstimated without new mesh %1.3f'%(a))
b = ((NPoints+1)**3)*(np.mean(IterationTimes)+np.mean(IterationTime))/(3600*24)
print('TotalTimeEstimated with new mesh %1.3f'%(b))

In [ ]:
NPoints = 10

# Optimization Parameters
ConstitutiveModels = ['Ogden', 'Neo-Hookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Fixed']
BCsType  = BCsTypes[1]


NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[0]

# Folder for the results
Output_Path = os.path.join('CostFunctionShape', BCsType, ConstitutiveModel)
os.makedirs(Output_Path, exist_ok=True)

FileName = str(NumberElements) + 'Elements' + str(NPoints) + '³Points.csv'

# Cost Function Data
CostData = pd.read_csv(os.path.join(Output_Path, FileName), sep=',', decimal='.')

# Row with the minimum cost
print('Min cost point for:')
print(CostData.loc[CostData['TotalCost'].idxmin()])

In [ ]:
# 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus    = CostData.Nu
Mus    = CostData.Mu
Alphas = CostData.Alpha
Costs  = CostData.TotalCost

Threshold = 1

Vmin = min(Costs)
Vmax = Threshold * (max(Costs) - min(Costs))

#img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[CostData['TotalCost'].idxmin()]], [Mus[CostData['TotalCost'].idxmin()]], [Alphas[CostData['TotalCost'].idxmin()]], marker = 'o', color='r')
plt.show()

In [ ]:
# Filtering

MaxRow = CostData['TotalCost'].idxmax()

Threshold = 0.00005
MaxCost = max(CostData.TotalCost)

FilteredCost = pd.DataFrame(columns=CostData.columns)

for Point in range(len(CostData)):
    CostValue = CostData.TotalCost[Point]
    if CostValue < MaxCost * Threshold:
        FilteredCost = FilteredCost.append(CostData.loc[Point])
        
# Filtered 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus    = FilteredCost.Nu
Mus    = FilteredCost.Mu
Alphas = FilteredCost.Alpha
Costs  = FilteredCost.TotalCost

#img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[CostData['TotalCost'].idxmin()]], [Mus[CostData['TotalCost'].idxmin()]], [Alphas[CostData['TotalCost'].idxmin()]], marker = 'o', color='r')
plt.show()

In [ ]:
# 2D plot
%matplotlib widget

Xaxes = ['Nu', 'Mu', 'Alpha']
Yaxes = ['Nu', 'Mu', 'Alpha']


# Plot tuning
Xaxis = Xaxes[0]
Yaxis = Yaxes[1]
Threshold = 0.0001


# Plot
Vmin = min(CostData.TotalCost)
Vmax = Threshold * (max(CostData.TotalCost) - min(CostData.TotalCost))

MinPoint = CostData['TotalCost'].idxmin()

CostData.plot.scatter(x=Xaxis,y=Yaxis,c='TotalCost', colormap='viridis', vmin=Vmin, vmax=Vmax, marker = '.')
plt.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', color='r', markersize=10, fillstyle='none')
plt.show()